<a href="https://colab.research.google.com/github/premkyumnam/demo-repo/blob/autoencoder.ipnyb/text_generation%20Using%20tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Poem text generator.ipynb
Automatically generated by Colaboratory.
# Text Generation using LSTM
##Two types of text generation
1.   Character based text generation
* Each character of the text is used to train the model and prediction will also result in generation of new characters. 
2.   Word based text generation
* Words are converted into tokens which are used to train the model. The model will generate words instead of characters in the prediction stage.
## Mechanics of the text generation model: 
1. The next word of the sequence is predicted using the words that are already present in the sequence. 
2. It is a simple model where splitting the data into training and testing sets is not required. This is because the model will use all the words in the sequence to predict the next word. Just like forecasting.
#The flow of program:
1. Loading data
2. Preprocessing the data and Tokenizing 
3. Building and fitting the model on data
4. Evaluate the model
5. Predicting(Generating the text)
6. Saving the model for future applicaitons
"""

In [ ]:
# mounting the device to read the text

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [ ]:
# reading the data from local google drive 
with open('/content/drive/My Drive/poem.txt') as story:
  story_data = story.read()

print(story_data)

In [ ]:
# data cleaning process
import re                                # Regular expressions to use sub function for replacing the useless text from the data


In [ ]:
def clean_text(text):
  text = re.sub(r',', '', text)
  text = re.sub(r'\'', '',  text)
  text = re.sub(r'\"', '', text)
  text = re.sub(r'\(', '', text)
  text = re.sub(r'\)', '', text)
  text = re.sub(r'\n', '', text)
  text = re.sub(r'“', '', text)
  text = re.sub(r'”', '', text)
  text = re.sub(r'’', '', text)
  text = re.sub(r'\.', '', text)
  text = re.sub(r';', '', text)
  text = re.sub(r':', '', text)
  text = re.sub(r'\-', '', text)

  return text

In [ ]:
# cleaning the data
lower_data = story_data.lower()           # Converting the string to lower case to get uniformity

split_data = lower_data.splitlines()      # Splitting the data to get every line seperately but this will give the list of uncleaned data

print(split_data)                         

In [ ]:
final = ''                                # initiating a argument with blank string to hold the values of final cleaned data


In [ ]:
for line in split_data:
  line = clean_text(line)
  final += '\n' + line

print(final)

In [ ]:
final_data = final.split('\n')       # splitting again to get list of cleaned and splitted data ready to be processed
print(final_data)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Instantiating the Tokenizer
max_vocab = 1000000
tokenizer = Tokenizer(num_words=max_vocab)
tokenizer.fit_on_texts(final_data)


In [ ]:
# Getting the total number of words of the data.
word2idx = tokenizer.word_index
print(len(word2idx))
print(word2idx)
vocab_size = len(word2idx) + 1        # Adding 1 to the vocab_size because the index starts from 1 not 0. This will make it uniform when using it further
print(vocab_size)


In [ ]:
"""## Creating n-gram sequences from the sentences
* Consider this sentence : ['two roads diverged in a yellow wood']. Here we will use ['two roads diverged in a yellow'] to predict ['wood']. This is the basic concept of forecasting which can be applied here to generate text.
* An advacement of this will be to use single word or every combination words possible from the sentence to predict the next word. And this is loosely termed as n_gram sequences
* The sentence ['two roads diverged in a yellow wood'] will have sequence as [112, 113, 114, 7, 5, 190, 75]
* so we will use combinations of words to make our model better
* [112, 113], 
* [112, 113, 114], 
* [112, 113, 114, 7], 
* [112, 113, 114, 7, 5], 
* [112, 113, 114, 7, 5, 190], 
* [112, 113, 114, 7, 5, 190, 75]
* we train our model that if 112 comes then it has to predict 113.
* if combination of 112, 113, comes then it has to predict 114 and so on.
"""

In [ ]:
# We will turn the sentences to sequences line by line and create n_gram sequences

input_seq = []

for line in final_data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_seq = token_list[:i+1]
    input_seq.append(n_gram_seq)

print(input_seq)

In [ ]:
# Getting the maximum length of sequence for padding purpose
max_seq_length = max(len(x) for x in input_seq)
print(max_seq_length)


In [ ]:
# Padding the sequences and converting them to array
input_seq = np.array(pad_sequences(input_seq, maxlen=max_seq_length, padding='pre'))
print(input_seq)

In [ ]:
#Taking xs and labels to train the model
xs = input_seq[:, :-1]        # xs contains every word in sentence except the last one because we are using this value to predict the y value
labels = input_seq[:, -1]     # labels contains only the last word of the sentence which will help in hot encoding the y value in next step
print("xs: ",xs)
print("labels:",labels)

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# one-hot encoding the labels according to the vocab size

# The matrix is square matrix of the size of vocab_size. Each row will denote a label and it will have 
# a single +ve value(i.e 1) for that label and other values will be zero. 

ys = to_categorical(labels, num_classes=vocab_size)
print(ys)

In [ ]:
from tensorflow.keras.layers import Input, Dense, Embedding, LSTM, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential


In [ ]:
# using the functional APIs of keras to define the model

i = Input(shape=(max_seq_length - 1, ))                           # using 1 less value becasuse we are preserving the last value for predicted word 
x = Embedding(vocab_size, 124)(i)
x = Dropout(0.2)(x)
x = LSTM(520, return_sequences=True)(x)
x = Bidirectional(layer=LSTM(340, return_sequences=True))(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(vocab_size, activation='softmax')(x)

model = Model(i,x)

In [ ]:
# using the pipeline method of sequential to define a model

# model = Sequential()
# model.add(Embedding(vocab_size, 124, input_length=max_seq_length-1))
# model.add(Dropout(0.2))
# model.add(LSTM(520, return_sequences=True))
# model.add(Bidirectional(LSTM(340, return_sequences=True)))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(1024, activation='relu'))
# model.add(Dense(vocab_size, activation='softmax'))

model.compile(optimizer=Adam(lr=0.001),
              loss = 'categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# model.summary()                                       # We can know about the shape of the model

r = model.fit(xs,ys,epochs=100)

In [ ]:
# Evaluating the model on accuracy
import matplotlib.pyplot as plt
plt.plot(r.history['accuracy'])

In [ ]:
# Defining a function to take input of seed text from user and no. of words to be predicted

def predict_words(seed, no_words):
  for i in range(no_words):
    token_list = tokenizer.texts_to_sequences([seed])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_length-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=1)


In [ ]:
new_word=''

In [ ]:
# predicting or generating the poem with the seed text

seed_text = 'i am feeling good today'
next_words = 20

predict_words(seed_text, next_words)

In [ ]:
# Saving the model
model.save('poem_generator.h5')  # will create a HDFS file of the model

In [ ]:
new_word